##IMPORTS

In [0]:
#installing py2neo driver for building connection with sandbox
!pip install py2neo

     |████████████████████████████████| 177 kB 5.1 MB/s 
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.


In [0]:
import py2neo
from py2neo import Graph
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, FloatType
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from pyspark.sql.types import ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import col,array_contains, map_from_entries, collect_list, struct
from pyspark.sql.functions import *
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import MinMaxScaler
from pyspark.sql import functions as f

In [0]:
spark = SparkSession.builder.appName('PySpark Read CSV').getOrCreate()

## LOADING HOMELOAN_CSV

In [0]:
df_home_loan = spark.read.options(header='True', delimiter=',') \
              .csv("/FileStore/tables/home_loan_data_draft_latest.csv")
display(df_home_loan)

customer_id,home_loan_id,tenure,orig_amt,rate,dpd,ltv,soi_cohort_employee,soi_cohort_online_heavy,soi_cohort_workplace,soi_cohort_millitary_banking,soi_cohort_student,orig_dt,maturity_dt,sch_amt,amount_owed,loan_status
CUST235925,HL868849,25,341897,0.090291502,DPD_0,0.201491365,NA,88,68,87,40,29-07-2008,28-06-2033,1152.61,0,InForce
CUST290194,HL537584,25,406327,0.175185036,DPD_0,1.014000854,NA,47,NA,NA,NA,27-03-2015,24-02-2040,1384.4,0,InForce
CUST288657,HL464229,12,327103,0.182285179,DPD_0,1.485103614,62,NA,99,57,52,25-05-2016,05-10-2028,2296.66,0,InForce
CUST301895,HL527310,25,267019,0.111554415,DPD_0,0.992136285,NA,NA,78,NA,NA,06-08-2015,07-05-2040,902.57,0,InForce
CUST747474,HL159536,10,89522,0.198102549,DPD_0,0.810476429,NA,NA,77,55,NA,12-11-2017,31-10-2027,753.49,0,InForce
CUST411524,HL898643,25,339070,0.183737854,DPD_0,0.661556655,NA,NA,NA,98,90,16-09-2010,16-08-2035,1156.48,0,InForce
CUST643840,HL862570,12,412542,0.061235838,DPD_0,1.40144415,50,49,77,71,69,10-05-2011,25-04-2023,2875.49,0,InForce
CUST472418,HL208697,25,484532,0.197888007,DPD_0,0.165780049,83,55,51,NA,81,12-07-2010,06-11-2035,1655.52,0,InForce
CUST164089,HL681200,20,84221,0.136640174,DPD_0,0.166789872,63,81,94,96,NA,07-11-2011,13-10-2031,355.76,0,InForce
CUST850621,HL805092,15,493150,0.137892239,DPD_0,0.355470777,NA,NA,57,NA,NA,07-11-2009,19-10-2024,2768.31,0,InForce


In [0]:
df_home_loan = df_home_loan.withColumn("tenure", df_home_loan["tenure"].cast(IntegerType()))
df_home_loan = df_home_loan.withColumn("orig_amt", df_home_loan["orig_amt"].cast(FloatType()))
df_home_loan = df_home_loan.withColumn("rate", df_home_loan["rate"].cast(FloatType()))
df_home_loan = df_home_loan.withColumn("ltv", df_home_loan["ltv"].cast(IntegerType()))
df_home_loan = df_home_loan.withColumn("sch_amt", df_home_loan["sch_amt"].cast(FloatType()))
df_home_loan = df_home_loan.withColumn("amount_owed",df_home_loan["amount_owed"].cast(FloatType()))
df_home_loan

Out[5]: DataFrame[customer_id: string, home_loan_id: string, tenure: int, orig_amt: float, rate: float, dpd: string, ltv: int, soi_cohort_employee: string, soi_cohort_online_heavy: string, soi_cohort_workplace: string, soi_cohort_millitary_banking: string, soi_cohort_student: string, orig_dt: string, maturity_dt: string, sch_amt: float, amount_owed: float, loan_status: string]

## LOADING CUSTOMER_CSV

In [0]:
df_customer = spark.read.options(header='True', delimiter=',') \
              .csv("/FileStore/tables/customer_23_12_latest.csv")
display(df_customer)

customer_id,age,gender,fico_scores,wallet_share,income_estimate,dti_ratio,average_time_spent(minutes),last_login,phone,ssn,action_performed,fname,lname,email,zip
CUST301488,36,M,428,0.214416597,112882,0.874758432,81,08-07-2017 23:02,7793364908,8887475444,Pay with Tap,James,Rogers,james.rogers@olson.com,112259
CUST235925,36,F,564,0.160260391,101331,0.670255106,48,27-12-2015 04:12,2377129120,7323084732,Pay with Tap,Margaret,Vasquez,margaret.vasquez@day.com,112263
CUST768307,41,M,743,0.080028863,97891,0.663282306,80,15-08-2013 10:16,4589178568,1425823974,Zelle QR Code,Derek,Rivera,derek.rivera@greene.com,112256
CUST854589,18,F,702,0.941966643,112343,0.429730273,39,29-01-2013 18:17,650494074,8475385450,Help Center,Sarah,Campbell,sarah.campbell@reese.com,112257
CUST288657,22,F,649,0.411026145,113523,0.626941097,15,08-04-2019 08:18,9728883573,267725099,Pay with Tap,Amanda,Lara,amanda.lara@downs-young.com,112259
CUST301895,35,F,315,0.992088335,97164,0.790688528,20,02-04-2011 04:59,5236363153,6809263436,Bill Payments,Jennifer,Jones,jennifer.jones@taylor-anderson.com,112262
CUST747228,28,F,724,0.873145174,108038,0.092366208,62,11-04-2018 18:35,8995232999,1918835875,Transaction Statements,Beverly,Gonzalez,beverly.gonzalez@thomas.com,112256
CUST626632,45,F,645,0.771350124,94656,0.939647569,36,15-08-2012 02:27,2333931362,2295650717,Zelle QR Code,Lisa,Gibbs,lisa.gibbs@townsend.com,112255
CUST648137,54,F,470,0.898435692,112199,0.670086718,15,20-12-2011 07:34,5916176340,3581714686,Help Center,Kiara,Duarte,kiara.duarte@bradford.org,112260
CUST499509,36,M,430,0.102420743,111066,0.977855526,24,26-07-2019 23:22,955885263,7099737570,Help Center,John,Lawson,john.lawson@allen.com,112256


In [0]:
df_customer = df_customer.withColumn("age", df_customer["age"].cast(IntegerType()))
df_customer = df_customer.withColumn("fico_scores", df_customer["fico_scores"].cast(IntegerType()))
df_customer = df_customer.withColumn("wallet_share", df_customer["wallet_share"].cast(FloatType()))
df_customer = df_customer.withColumn("income_estimate", df_customer["income_estimate"].cast(IntegerType()))
df_customer = df_customer.withColumn("dti_ratio", df_customer["dti_ratio"].cast(FloatType()))
df_customer = df_customer.withColumn("average_time_spent(minutes)",df_customer["average_time_spent(minutes)"].cast(IntegerType()))
df_customer = df_customer.withColumn("zip",df_customer["zip"].cast(IntegerType()))
df_customer

Out[7]: DataFrame[customer_id: string, age: int, gender: string, fico_scores: int, wallet_share: float, income_estimate: int, dti_ratio: float, average_time_spent(minutes): int, last_login: string, phone: string, ssn: string, action_performed: string, fname: string, lname: string, email: string, zip: int]

## Merging Customer and HomeLoan Data

In [0]:
merged_customer_home_loan = df_customer.join(df_home_loan,df_customer["customer_id"] == df_home_loan["customer_id"],'left').drop(df_home_loan.customer_id)
display(merged_customer_home_loan)

customer_id,age,gender,fico_scores,wallet_share,income_estimate,dti_ratio,average_time_spent(minutes),last_login,phone,ssn,action_performed,fname,lname,email,zip,home_loan_id,tenure,orig_amt,rate,dpd,ltv,soi_cohort_employee,soi_cohort_online_heavy,soi_cohort_workplace,soi_cohort_millitary_banking,soi_cohort_student,orig_dt,maturity_dt,sch_amt,amount_owed,loan_status
CUST301488,36,M,428,0.2144166,112882,0.8747584,81,08-07-2017 23:02,7793364908,8887475444,Pay with Tap,James,Rogers,james.rogers@olson.com,112259,HL199671,25,293295.0,0.19137831,DPD_0,0,58,NA,NA,NA,NA,05-02-2015,01-05-2040,1001.3,0.0,InForce
CUST235925,36,F,564,0.1602604,101331,0.6702551,48,27-12-2015 04:12,2377129120,7323084732,Pay with Tap,Margaret,Vasquez,margaret.vasquez@day.com,112263,HL868849,25,341897.0,0.0902915,DPD_0,0,NA,88,68,87,40,29-07-2008,28-06-2033,1152.61,0.0,InForce
CUST768307,41,M,743,0.08002886,97891,0.66328233,80,15-08-2013 10:16,4589178568,1425823974,Zelle QR Code,Derek,Rivera,derek.rivera@greene.com,112256,HL163604,10,417023.0,0.10936252,DPD_0,0,NA,68,NA,84,NA,26-01-2011,13-01-2021,3494.39,0.0,Matured
CUST854589,18,F,702,0.94196665,112343,0.42973027,39,29-01-2013 18:17,650494074,8475385450,Help Center,Sarah,Campbell,sarah.campbell@reese.com,112257,HL122253,12,142169.0,0.0677801,DPD_0,1,NA,NA,60,NA,74,02-03-2009,15-02-2021,991.33,0.0,Matured
CUST288657,22,F,649,0.41102615,113523,0.6269411,15,08-04-2019 08:18,9728883573,267725099,Pay with Tap,Amanda,Lara,amanda.lara@downs-young.com,112259,HL464229,12,327103.0,0.18228517,DPD_0,1,62,NA,99,57,52,25-05-2016,05-10-2028,2296.66,0.0,InForce
CUST301895,35,F,315,0.9920883,97164,0.7906885,20,02-04-2011 04:59,5236363153,6809263436,Bill Payments,Jennifer,Jones,jennifer.jones@taylor-anderson.com,112262,HL527310,25,267019.0,0.111554414,DPD_0,0,NA,NA,78,NA,NA,06-08-2015,07-05-2040,902.57,0.0,InForce
CUST747228,28,F,724,0.87314516,108038,0.09236621,62,11-04-2018 18:35,8995232999,1918835875,Transaction Statements,Beverly,Gonzalez,beverly.gonzalez@thomas.com,112256,HL872988,15,385598.0,0.080322176,DPD_0,1,NA,92,NA,70,NA,21-07-2019,07-02-2034,2155.21,0.0,InForce
CUST626632,45,F,645,0.77135015,94656,0.93964756,36,15-08-2012 02:27,2333931362,2295650717,Zelle QR Code,Lisa,Gibbs,lisa.gibbs@townsend.com,112255,HL195807,10,490995.0,0.15851834,DPD_0,0,NA,NA,NA,89,NA,10-11-2018,28-10-2028,4124.41,0.0,InForce
CUST648137,54,F,470,0.8984357,112199,0.67008674,15,20-12-2011 07:34,5916176340,3581714686,Help Center,Kiara,Duarte,kiara.duarte@bradford.org,112260,HL431682,25,350086.0,0.090155594,DPD_0,0,NA,NA,NA,40,99,21-04-2018,21-03-2043,1180.2,0.0,InForce
CUST499509,36,M,430,0.10242074,111066,0.9778555,24,26-07-2019 23:22,955885263,7099737570,Help Center,John,Lawson,john.lawson@allen.com,112256,HL560265,12,73593.0,0.19339575,DPD_90,0,79,64,64,69,54,12-02-2012,28-01-2024,517.06,1551.18,InForce


## Making Target Variable - Home Loan Defaulters

In [0]:
merged_customer_home_loan = merged_customer_home_loan.withColumn("home_loan_defaulters", when(merged_customer_home_loan.dpd == 'DPD_0', 0).otherwise(1))
display(merged_customer_home_loan)

customer_id,age,gender,fico_scores,wallet_share,income_estimate,dti_ratio,average_time_spent(minutes),last_login,phone,ssn,action_performed,fname,lname,email,zip,home_loan_id,tenure,orig_amt,rate,dpd,ltv,soi_cohort_employee,soi_cohort_online_heavy,soi_cohort_workplace,soi_cohort_millitary_banking,soi_cohort_student,orig_dt,maturity_dt,sch_amt,amount_owed,loan_status,home_loan_defaulters
CUST301488,36,M,428,0.2144166,112882,0.8747584,81,08-07-2017 23:02,7793364908,8887475444,Pay with Tap,James,Rogers,james.rogers@olson.com,112259,HL199671,25,293295.0,0.19137831,DPD_0,0,58,NA,NA,NA,NA,05-02-2015,01-05-2040,1001.3,0.0,InForce,0
CUST235925,36,F,564,0.1602604,101331,0.6702551,48,27-12-2015 04:12,2377129120,7323084732,Pay with Tap,Margaret,Vasquez,margaret.vasquez@day.com,112263,HL868849,25,341897.0,0.0902915,DPD_0,0,NA,88,68,87,40,29-07-2008,28-06-2033,1152.61,0.0,InForce,0
CUST768307,41,M,743,0.08002886,97891,0.66328233,80,15-08-2013 10:16,4589178568,1425823974,Zelle QR Code,Derek,Rivera,derek.rivera@greene.com,112256,HL163604,10,417023.0,0.10936252,DPD_0,0,NA,68,NA,84,NA,26-01-2011,13-01-2021,3494.39,0.0,Matured,0
CUST854589,18,F,702,0.94196665,112343,0.42973027,39,29-01-2013 18:17,650494074,8475385450,Help Center,Sarah,Campbell,sarah.campbell@reese.com,112257,HL122253,12,142169.0,0.0677801,DPD_0,1,NA,NA,60,NA,74,02-03-2009,15-02-2021,991.33,0.0,Matured,0
CUST288657,22,F,649,0.41102615,113523,0.6269411,15,08-04-2019 08:18,9728883573,267725099,Pay with Tap,Amanda,Lara,amanda.lara@downs-young.com,112259,HL464229,12,327103.0,0.18228517,DPD_0,1,62,NA,99,57,52,25-05-2016,05-10-2028,2296.66,0.0,InForce,0
CUST301895,35,F,315,0.9920883,97164,0.7906885,20,02-04-2011 04:59,5236363153,6809263436,Bill Payments,Jennifer,Jones,jennifer.jones@taylor-anderson.com,112262,HL527310,25,267019.0,0.111554414,DPD_0,0,NA,NA,78,NA,NA,06-08-2015,07-05-2040,902.57,0.0,InForce,0
CUST747228,28,F,724,0.87314516,108038,0.09236621,62,11-04-2018 18:35,8995232999,1918835875,Transaction Statements,Beverly,Gonzalez,beverly.gonzalez@thomas.com,112256,HL872988,15,385598.0,0.080322176,DPD_0,1,NA,92,NA,70,NA,21-07-2019,07-02-2034,2155.21,0.0,InForce,0
CUST626632,45,F,645,0.77135015,94656,0.93964756,36,15-08-2012 02:27,2333931362,2295650717,Zelle QR Code,Lisa,Gibbs,lisa.gibbs@townsend.com,112255,HL195807,10,490995.0,0.15851834,DPD_0,0,NA,NA,NA,89,NA,10-11-2018,28-10-2028,4124.41,0.0,InForce,0
CUST648137,54,F,470,0.8984357,112199,0.67008674,15,20-12-2011 07:34,5916176340,3581714686,Help Center,Kiara,Duarte,kiara.duarte@bradford.org,112260,HL431682,25,350086.0,0.090155594,DPD_0,0,NA,NA,NA,40,99,21-04-2018,21-03-2043,1180.2,0.0,InForce,0
CUST499509,36,M,430,0.10242074,111066,0.9778555,24,26-07-2019 23:22,955885263,7099737570,Help Center,John,Lawson,john.lawson@allen.com,112256,HL560265,12,73593.0,0.19339575,DPD_90,0,79,64,64,69,54,12-02-2012,28-01-2024,517.06,1551.18,InForce,1


## Feature Engineering - Scaling Features for GDS Algorithms

In [0]:
columns_to_scale = ["age", "fico_scores", "wallet_share","income_estimate","dti_ratio","tenure","orig_amt","rate","sch_amt"]
assemblers = [VectorAssembler(inputCols=[col], outputCol=col + "_vec") for col in columns_to_scale]
scalers = [MinMaxScaler(inputCol=col + "_vec", outputCol=col + "_scaled") for col in columns_to_scale]
pipeline = Pipeline(stages=assemblers + scalers)
scalerModel = pipeline.fit(merged_customer_home_loan.select(columns_to_scale))
vectorized_scaled_data = scalerModel.transform(merged_customer_home_loan.select(columns_to_scale))
display(vectorized_scaled_data)

age,fico_scores,wallet_share,income_estimate,dti_ratio,tenure,orig_amt,rate,sch_amt,age_vec,fico_scores_vec,wallet_share_vec,income_estimate_vec,dti_ratio_vec,tenure_vec,orig_amt_vec,rate_vec,sch_amt_vec,age_scaled,fico_scores_scaled,wallet_share_scaled,income_estimate_scaled,dti_ratio_scaled,tenure_scaled,orig_amt_scaled,rate_scaled,sch_amt_scaled
36,428,0.2144166,112882,0.8747584,25,293295.0,0.19137831,1001.3,"Map(vectorType -> dense, length -> 1, values -> List(36.0))","Map(vectorType -> dense, length -> 1, values -> List(428.0))","Map(vectorType -> dense, length -> 1, values -> List(0.21441659331321716))","Map(vectorType -> dense, length -> 1, values -> List(112882.0))","Map(vectorType -> dense, length -> 1, values -> List(0.8747584223747253))","Map(vectorType -> dense, length -> 1, values -> List(25.0))","Map(vectorType -> dense, length -> 1, values -> List(293295.0))","Map(vectorType -> dense, length -> 1, values -> List(0.19137831032276154))","Map(vectorType -> dense, length -> 1, values -> List(1001.2999877929688))","Map(vectorType -> dense, length -> 1, values -> List(0.4))","Map(vectorType -> dense, length -> 1, values -> List(0.18311874105865522))","Map(vectorType -> dense, length -> 1, values -> List(0.19321811614351062))","Map(vectorType -> dense, length -> 1, values -> List(0.04561629633780717))","Map(vectorType -> dense, length -> 1, values -> List(0.8849990407675538))","Map(vectorType -> dense, length -> 1, values -> List(1.0))","Map(vectorType -> dense, length -> 1, values -> List(0.5237797071762761))","Map(vectorType -> dense, length -> 1, values -> List(0.9417527393416383))","Map(vectorType -> dense, length -> 1, values -> List(0.1712920947648968))"
36,564,0.1602604,101331,0.6702551,25,341897.0,0.0902915,1152.61,"Map(vectorType -> dense, length -> 1, values -> List(36.0))","Map(vectorType -> dense, length -> 1, values -> List(564.0))","Map(vectorType -> dense, length -> 1, values -> List(0.1602603942155838))","Map(vectorType -> dense, length -> 1, values -> List(101331.0))","Map(vectorType -> dense, length -> 1, values -> List(0.6702551245689392))","Map(vectorType -> dense, length -> 1, values -> List(25.0))","Map(vectorType -> dense, length -> 1, values -> List(341897.0))","Map(vectorType -> dense, length -> 1, values -> List(0.09029150009155273))","Map(vectorType -> dense, length -> 1, values -> List(1152.6099853515625))","Map(vectorType -> dense, length -> 1, values -> List(0.4))","Map(vectorType -> dense, length -> 1, values -> List(0.37768240343347637))","Map(vectorType -> dense, length -> 1, values -> List(0.13703471773676548))","Map(vectorType -> dense, length -> 1, values -> List(0.02071959254844738))","Map(vectorType -> dense, length -> 1, values -> List(0.6764196104469115))","Map(vectorType -> dense, length -> 1, values -> List(1.0))","Map(vectorType -> dense, length -> 1, values -> List(0.6369102208980237))","Map(vectorType -> dense, length -> 1, values -> List(0.2446348107531531))","Map(vectorType -> dense, length -> 1, values -> List(0.21144174826875012))"
41,743,0.08002886,97891,0.66328233,10,417023.0,0.10936252,3494.39,"Map(vectorType -> dense, length -> 1, values -> List(41.0))","Map(vectorType -> dense, length -> 1, values -> List(743.0))","Map(vectorType -> dense, length -> 1, values -> List(0.08002886176109314))","Map(vectorType -> dense, length -> 1, values -> List(97891.0))","Map(vectorType -> dense, length -> 1, values -> List(0.6632823348045349))","Map(vectorType -> dense, length -> 1, values -> List(10.0))","Map(vectorType -> dense, length -> 1, values -> List(417023.0))","Map(vectorType -> dense, length -> 1, values -> List(0.10936252027750015))","Map(vectorType -> dense, length -> 1, values -> List(3494.389892578125))","Map(vectorType -> dense, length -> 1, values -> List(0.5111111111111112))","Map(vectorType -> dense, length -> 1, values -> List(0.6337625178826896))","Map(vectorType -> dense, length -> 1, values -> List(0.053799922315693555))","Map(vectorType -> dense, length -> 1,

In [0]:
scaled_columns = [f for f in vectorized_scaled_data.columns if '_scaled' in f]
scaled_columns

Out[12]: ['age_scaled',
 'fico_scores_scaled',
 'wallet_share_scaled',
 'income_estimate_scaled',
 'dti_ratio_scaled',
 'tenure_scaled',
 'orig_amt_scaled',
 'rate_scaled',
 'sch_amt_scaled']

## Changing vectorized columns into single value columnar format

In [0]:
firstelement=f.udf(lambda v:float(v[0]),FloatType())
scaled_data = vectorized_scaled_data.select([firstelement(c).alias(c) for c in scaled_columns])
display(scaled_data)

age_scaled,fico_scores_scaled,wallet_share_scaled,income_estimate_scaled,dti_ratio_scaled,tenure_scaled,orig_amt_scaled,rate_scaled,sch_amt_scaled
0.4,0.18311875,0.19321811,0.045616295,0.88499904,1.0,0.5237797,0.94175273,0.1712921
0.4,0.37768242,0.13703471,0.020719593,0.6764196,1.0,0.6369102,0.2446348,0.21144176
0.51111114,0.63376254,0.053799924,0.013305113,0.6693078,0.0,0.81178045,0.37615296,0.832826
0.0,0.5751073,0.94800216,0.044454552,0.43110064,0.13333334,0.17200485,0.08939101,0.16864659
0.08888889,0.49928468,0.39718726,0.04699789,0.63224226,0.13333334,0.60247433,0.87904435,0.51501197
0.37777779,0.021459227,1.0,0.011738157,0.79925346,1.0,0.46261725,0.39126876,0.1450944
0.22222222,0.60658085,0.8766045,0.035175674,0.08701229,0.33333334,0.73863274,0.17588405,0.47747865
0.6,0.49356222,0.7709991,0.0063324836,0.95118153,0.0,0.9839645,0.71514285,1.0
0.8,0.24320458,0.90284175,0.044144176,0.6762479,1.0,0.6559717,0.24369757,0.21876267
0.4,0.18597998,0.07702998,0.04170214,0.99015105,0.13333334,0.012380997,0.95566547,0.042800467


## Merging Scaled Features with Customer_Home_Loan merged data

In [0]:
from pyspark.sql.functions import monotonically_increasing_id
merged_customer_home_loan = merged_customer_home_loan.withColumn("id",monotonically_increasing_id())
scaled_data = scaled_data.withColumn("id",monotonically_increasing_id())
merged_customer_home_loan_scaled_data = merged_customer_home_loan.join(scaled_data,merged_customer_home_loan["id"] == scaled_data["id"],'inner').drop(scaled_data.id).drop(merged_customer_home_loan.id)
display(merged_customer_home_loan_scaled_data)

customer_id,age,gender,fico_scores,wallet_share,income_estimate,dti_ratio,average_time_spent(minutes),last_login,phone,ssn,action_performed,fname,lname,email,zip,home_loan_id,tenure,orig_amt,rate,dpd,ltv,soi_cohort_employee,soi_cohort_online_heavy,soi_cohort_workplace,soi_cohort_millitary_banking,soi_cohort_student,orig_dt,maturity_dt,sch_amt,amount_owed,loan_status,home_loan_defaulters,age_scaled,fico_scores_scaled,wallet_share_scaled,income_estimate_scaled,dti_ratio_scaled,tenure_scaled,orig_amt_scaled,rate_scaled,sch_amt_scaled
CUST301488,36,M,428,0.2144166,112882,0.8747584,81,08-07-2017 23:02,7793364908,8887475444,Pay with Tap,James,Rogers,james.rogers@olson.com,112259,HL199671,25,293295.0,0.19137831,DPD_0,0,58,NA,NA,NA,NA,05-02-2015,01-05-2040,1001.3,0.0,InForce,0,0.4,0.18311875,0.19321811,0.045616295,0.88499904,1.0,0.5237797,0.94175273,0.1712921
CUST235925,36,F,564,0.1602604,101331,0.6702551,48,27-12-2015 04:12,2377129120,7323084732,Pay with Tap,Margaret,Vasquez,margaret.vasquez@day.com,112263,HL868849,25,341897.0,0.0902915,DPD_0,0,NA,88,68,87,40,29-07-2008,28-06-2033,1152.61,0.0,InForce,0,0.4,0.37768242,0.13703471,0.020719593,0.6764196,1.0,0.6369102,0.2446348,0.21144176
CUST768307,41,M,743,0.08002886,97891,0.66328233,80,15-08-2013 10:16,4589178568,1425823974,Zelle QR Code,Derek,Rivera,derek.rivera@greene.com,112256,HL163604,10,417023.0,0.10936252,DPD_0,0,NA,68,NA,84,NA,26-01-2011,13-01-2021,3494.39,0.0,Matured,0,0.51111114,0.63376254,0.053799924,0.013305113,0.6693078,0.0,0.81178045,0.37615296,0.832826
CUST854589,18,F,702,0.94196665,112343,0.42973027,39,29-01-2013 18:17,650494074,8475385450,Help Center,Sarah,Campbell,sarah.campbell@reese.com,112257,HL122253,12,142169.0,0.0677801,DPD_0,1,NA,NA,60,NA,74,02-03-2009,15-02-2021,991.33,0.0,Matured,0,0.0,0.5751073,0.94800216,0.044454552,0.43110064,0.13333334,0.17200485,0.08939101,0.16864659
CUST288657,22,F,649,0.41102615,113523,0.6269411,15,08-04-2019 08:18,9728883573,267725099,Pay with Tap,Amanda,Lara,amanda.lara@downs-young.com,112259,HL464229,12,327103.0,0.18228517,DPD_0,1,62,NA,99,57,52,25-05-2016,05-10-2028,2296.66,0.0,InForce,0,0.08888889,0.49928468,0.39718726,0.04699789,0.63224226,0.13333334,0.60247433,0.87904435,0.51501197
CUST301895,35,F,315,0.9920883,97164,0.7906885,20,02-04-2011 04:59,5236363153,6809263436,Bill Payments,Jennifer,Jones,jennifer.jones@taylor-anderson.com,112262,HL527310,25,267019.0,0.111554414,DPD_0,0,NA,NA,78,NA,NA,06-08-2015,07-05-2040,902.57,0.0,InForce,0,0.37777779,0.021459227,1.0,0.011738157,0.79925346,1.0,0.46261725,0.39126876,0.1450944
CUST747228,28,F,724,0.87314516,108038,0.09236621,62,11-04-2018 18:35,8995232999,1918835875,Transaction Statements,Beverly,Gonzalez,beverly.gonzalez@thomas.com,112256,HL872988,15,385598.0,0.080322176,DPD_0,1,NA,92,NA,70,NA,21-07-2019,07-02-2034,2155.21,0.0,InForce,0,0.22222222,0.60658085,0.8766045,0.035175674,0.08701229,0.33333334,0.73863274,0.17588405,0.47747865
CUST626632,45,F,645,0.77135015,94656,0.93964756,36,15-08-2012 02:27,2333931362,2295650717,Zelle QR Code,Lisa,Gibbs,lisa.gibbs@townsend.com,112255,HL195807,10,490995.0,0.15851834,DPD_0,0,NA,NA,NA,89,NA,10-11-2018,28-10-2028,4124.41,0.0,InForce,0,0.6,0.49356222,0.7709991,0.0063324836,0.95118153,0.0,0.9839645,0.71514285,1.0
CUST648137,54,F,470,0.8984357,112199,0.67008674,15,20-12-2011 07:34,5916176340,3581714686,Help Center,Kiara,Duarte,kiara.duarte@bradford.org,112260,HL431682,25,350086.0,0.090155594,DPD_0,0,NA,NA,NA,40,99,21-04-2018,21-03-2043,1180.2,0.0,InForce,0,0.8,0.24320458,0.90284175,0.044144176,0.6762479,1.0,0.6559717,0.24369757,0.21876267
CUST499509,36,M,430,0.10242074,111066,0.9778555,24,26-07-2019 23:22,955885263,7099737570,Help Center,John,Lawson,john.lawson@allen.com,112256,HL560265,12,73593.0,0.19339575,DPD_90,0,79,64,64,69,54,12-02-2012,28-01-2024,517.06,1551.18,InForce,1,0.4,0.18597998,0.07702998,0.04170214,0.99015105,0.13333334,0.012380997,0.95566547,0.042800467


## Mapping Gender categorical values to 0 and 1

In [0]:
merged_customer_home_loan_scaled_data = merged_customer_home_loan_scaled_data.withColumn("gender_mapped (M:0,F:1)", when(merged_customer_home_loan_scaled_data.gender == 'M', 0).otherwise(1))
display(merged_customer_home_loan_scaled_data)

customer_id,age,gender,fico_scores,wallet_share,income_estimate,dti_ratio,average_time_spent(minutes),last_login,phone,ssn,action_performed,fname,lname,email,zip,home_loan_id,tenure,orig_amt,rate,dpd,ltv,soi_cohort_employee,soi_cohort_online_heavy,soi_cohort_workplace,soi_cohort_millitary_banking,soi_cohort_student,orig_dt,maturity_dt,sch_amt,amount_owed,loan_status,home_loan_defaulters,age_scaled,fico_scores_scaled,wallet_share_scaled,income_estimate_scaled,dti_ratio_scaled,tenure_scaled,orig_amt_scaled,rate_scaled,sch_amt_scaled,"gender_mapped (M:0,F:1)"
CUST301488,36,M,428,0.2144166,112882,0.8747584,81,08-07-2017 23:02,7793364908,8887475444,Pay with Tap,James,Rogers,james.rogers@olson.com,112259,HL199671,25,293295.0,0.19137831,DPD_0,0,58,NA,NA,NA,NA,05-02-2015,01-05-2040,1001.3,0.0,InForce,0,0.4,0.18311875,0.19321811,0.045616295,0.88499904,1.0,0.5237797,0.94175273,0.1712921,0
CUST235925,36,F,564,0.1602604,101331,0.6702551,48,27-12-2015 04:12,2377129120,7323084732,Pay with Tap,Margaret,Vasquez,margaret.vasquez@day.com,112263,HL868849,25,341897.0,0.0902915,DPD_0,0,NA,88,68,87,40,29-07-2008,28-06-2033,1152.61,0.0,InForce,0,0.4,0.37768242,0.13703471,0.020719593,0.6764196,1.0,0.6369102,0.2446348,0.21144176,1
CUST768307,41,M,743,0.08002886,97891,0.66328233,80,15-08-2013 10:16,4589178568,1425823974,Zelle QR Code,Derek,Rivera,derek.rivera@greene.com,112256,HL163604,10,417023.0,0.10936252,DPD_0,0,NA,68,NA,84,NA,26-01-2011,13-01-2021,3494.39,0.0,Matured,0,0.51111114,0.63376254,0.053799924,0.013305113,0.6693078,0.0,0.81178045,0.37615296,0.832826,0
CUST854589,18,F,702,0.94196665,112343,0.42973027,39,29-01-2013 18:17,650494074,8475385450,Help Center,Sarah,Campbell,sarah.campbell@reese.com,112257,HL122253,12,142169.0,0.0677801,DPD_0,1,NA,NA,60,NA,74,02-03-2009,15-02-2021,991.33,0.0,Matured,0,0.0,0.5751073,0.94800216,0.044454552,0.43110064,0.13333334,0.17200485,0.08939101,0.16864659,1
CUST288657,22,F,649,0.41102615,113523,0.6269411,15,08-04-2019 08:18,9728883573,267725099,Pay with Tap,Amanda,Lara,amanda.lara@downs-young.com,112259,HL464229,12,327103.0,0.18228517,DPD_0,1,62,NA,99,57,52,25-05-2016,05-10-2028,2296.66,0.0,InForce,0,0.08888889,0.49928468,0.39718726,0.04699789,0.63224226,0.13333334,0.60247433,0.87904435,0.51501197,1
CUST301895,35,F,315,0.9920883,97164,0.7906885,20,02-04-2011 04:59,5236363153,6809263436,Bill Payments,Jennifer,Jones,jennifer.jones@taylor-anderson.com,112262,HL527310,25,267019.0,0.111554414,DPD_0,0,NA,NA,78,NA,NA,06-08-2015,07-05-2040,902.57,0.0,InForce,0,0.37777779,0.021459227,1.0,0.011738157,0.79925346,1.0,0.46261725,0.39126876,0.1450944,1
CUST747228,28,F,724,0.87314516,108038,0.09236621,62,11-04-2018 18:35,8995232999,1918835875,Transaction Statements,Beverly,Gonzalez,beverly.gonzalez@thomas.com,112256,HL872988,15,385598.0,0.080322176,DPD_0,1,NA,92,NA,70,NA,21-07-2019,07-02-2034,2155.21,0.0,InForce,0,0.22222222,0.60658085,0.8766045,0.035175674,0.08701229,0.33333334,0.73863274,0.17588405,0.47747865,1
CUST626632,45,F,645,0.77135015,94656,0.93964756,36,15-08-2012 02:27,2333931362,2295650717,Zelle QR Code,Lisa,Gibbs,lisa.gibbs@townsend.com,112255,HL195807,10,490995.0,0.15851834,DPD_0,0,NA,NA,NA,89,NA,10-11-2018,28-10-2028,4124.41,0.0,InForce,0,0.6,0.49356222,0.7709991,0.0063324836,0.95118153,0.0,0.9839645,0.71514285,1.0,1
CUST648137,54,F,470,0.8984357,112199,0.67008674,15,20-12-2011 07:34,5916176340,3581714686,Help Center,Kiara,Duarte,kiara.duarte@bradford.org,112260,HL431682,25,350086.0,0.090155594,DPD_0,0,NA,NA,NA,40,99,21-04-2018,21-03-2043,1180.2,0.0,InForce,0,0.8,0.24320458,0.90284175,0.044144176,0.6762479,1.0,0.6559717,0.24369757,0.21876267,1
CUST499509,36,M,430,0.10242074,111066,0.9778555,24,26-07-2019 23:22,955885263,7099737570,Help Center,John,Lawson,john.lawson@allen.com,112256,HL560265,12,73593.0,0.19339575,DPD_90,0,79,64,64,69,54,12-02-2012,28-01-2024,517.06,1551.18,InForce,1,0.4,0.18597998,0.07702998,0.04170214,0.99015105,0.13333334,0.012380997,0.95566547,0.042800467,0


In [0]:
defaulters_data = merged_customer_home_loan_scaled_data.where(merged_customer_home_loan_scaled_data.home_loan_defaulters == 1)
display(defaulters_data)

customer_id,age,gender,fico_scores,wallet_share,income_estimate,dti_ratio,average_time_spent(minutes),last_login,phone,ssn,action_performed,fname,lname,email,zip,home_loan_id,tenure,orig_amt,rate,dpd,ltv,soi_cohort_employee,soi_cohort_online_heavy,soi_cohort_workplace,soi_cohort_millitary_banking,soi_cohort_student,orig_dt,maturity_dt,sch_amt,amount_owed,loan_status,home_loan_defaulters,age_scaled,fico_scores_scaled,wallet_share_scaled,income_estimate_scaled,dti_ratio_scaled,tenure_scaled,orig_amt_scaled,rate_scaled,sch_amt_scaled,"gender_mapped (M:0,F:1)"
CUST499509,36,M,430,0.10242074,111066,0.9778555,24,26-07-2019 23:22,955885263,7099737570,Help Center,John,Lawson,john.lawson@allen.com,112256,HL560265,12,73593.0,0.19339575,DPD_90,0,79,64,64,69,54,12-02-2012,28-01-2024,517.06,1551.18,InForce,1,0.4,0.18597998,0.07702998,0.04170214,0.99015105,0.13333334,0.012380997,0.95566547,0.042800467,0
CUST594764,20,F,658,0.25493795,92632,0.8950112,37,10-01-2014 09:55,525476139,1037603771,Transfer Funds,Jocelyn,Perry,jocelyn.perry@walsh-huber.com,112261,HL426857,12,186185.0,0.08071548,DPD_90,0,NA,NA,NA,NA,NA,06-09-2009,22-08-2021,1299.27,3897.81,Overdue,1,0.044444446,0.51216024,0.23525628,0.00197001,0.9056555,0.13333334,0.27446055,0.17859635,0.25035754,1
CUST305070,55,M,366,0.5502413,116731,0.95298964,7,03-08-2009 03:38,2599887850,8773615865,Pay with Tap,Dale,Moore,dale.moore@gilbert.net,112260,HL551585,15,68274.0,0.11426555,DPD_30,0,87,76,52,92,78,28-11-2019,11-09-2034,382.58,382.58,InForce,1,0.82222223,0.094420604,0.5416136,0.053912323,0.96478957,0.33333334,0.0,0.4099654,0.0071166004,0
CUST768887,26,M,476,0.2493546,97241,0.25433972,30,14-03-2010 16:26,3910536420,5942038379,Bill Payments,Michael,Hall,michael.hall@garcia.com,112257,HL892732,15,71272.0,0.121932514,DPD_30,1,NA,89,NA,66,NA,28-10-2012,10-10-2027,399.61,399.61,InForce,1,0.17777778,0.25178826,0.22946393,0.01190412,0.25221425,0.33333334,0.0069784224,0.46283853,0.011635459,0
CUST219645,38,M,771,0.92347324,92494,0.16453096,89,15-03-2015 04:16,1596449396,3348344054,Transaction Statements,Patrick,Bruce,patrick.bruce@henson.org,112264,HL441567,20,350485.0,0.08045979,DPD_30,0,NA,78,65,94,87,16-01-2008,22-12-2027,1472.18,1472.18,InForce,1,0.44444445,0.6738197,0.9288165,0.0016725688,0.16061543,0.6666667,0.65690047,0.17683306,0.29623872,0
CUST220402,63,M,588,0.26589173,98649,0.54949677,42,19-05-2017 08:19,2706889691,9607922433,Customize Card,Sean,King,sean.king@dougherty.biz,112258,HL388771,12,116362.0,0.07188486,DPD_30,1,NA,NA,98,76,40,08-09-2008,24-08-2020,811.58,811.58,Overdue,1,1.0,0.41201717,0.24662009,0.014938884,0.5532543,0.13333334,0.11193408,0.1176984,0.12095047,0
CUST689781,43,F,697,0.70783985,96666,0.6768759,65,03-01-2014 05:44,5650582702,3826592144,Transaction Statements,Shari,Maldonado,shari.maldonado@cline.com,112261,HL243518,25,353168.0,0.098377995,DPD_60,0,47,83,NA,NA,NA,15-03-2008,02-12-2033,1191.81,2383.62,InForce,1,0.5555556,0.56795424,0.7051114,0.010664782,0.68317235,1.0,0.66314566,0.30040115,0.22184338,1
CUST316296,46,F,353,0.028170139,109881,0.75311524,17,06-07-2014 16:52,1222380877,310852864,Help Center,Regina,Robinson,regina.robinson@humphrey-brooks.com,112263,HL408872,10,182866.0,0.085013784,DPD_30,1,NA,NA,45,NA,NA,11-01-2013,30-12-2022,1530.42,1530.42,InForce,1,0.62222224,0.07582261,0.0,0.039148025,0.76093125,0.0,0.26673496,0.20823847,0.3116925,1
CUST537213,21,F,330,0.6880198,96310,0.5543166,86,11-01-2014 00:26,6970514612,838712666,Help Center,Bailey,West,bailey.west@valdez-blair.info,112260,HL139128,20,475770.0,0.10235902,DPD_60,0,NA,NA,46,58,NA,15-02-2009,21-01-2029,2002.82,4005.64,InForce,1,0.06666667,0.042918455,0.68454945,0.009897469,0.5581702,0.6666667,0.9485254,0.3278552,0.43704242,1
CUST8765,30,M,888,0.765425,555675,0.987512,76,20-12-2015 06:34,43214568,5678962134,Easy Lock,Thejas,Gurki,thejas.gurki@gmail.com,112262,HL567542,10,217865.0,0.0548178,DPD_30,0,NA,NA,65,NA,NA,18-04-2015,02-12-2025,2544.61,0.0,InForce,1,0.26666668,0.8412017,0.76485217,1.0,1.0,0.

In [0]:
defaulters_data = defaulters_data.withColumnRenamed("customer_id", "customer_id_defaulters")
display(defaulters_data)

customer_id_defaulters,age,gender,fico_scores,wallet_share,income_estimate,dti_ratio,average_time_spent(minutes),last_login,phone,ssn,action_performed,fname,lname,email,zip,home_loan_id,tenure,orig_amt,rate,dpd,ltv,soi_cohort_employee,soi_cohort_online_heavy,soi_cohort_workplace,soi_cohort_millitary_banking,soi_cohort_student,orig_dt,maturity_dt,sch_amt,amount_owed,loan_status,home_loan_defaulters,age_scaled,fico_scores_scaled,wallet_share_scaled,income_estimate_scaled,dti_ratio_scaled,tenure_scaled,orig_amt_scaled,rate_scaled,sch_amt_scaled,"gender_mapped (M:0,F:1)"
CUST499509,36,M,430,0.10242074,111066,0.9778555,24,26-07-2019 23:22,955885263,7099737570,Help Center,John,Lawson,john.lawson@allen.com,112256,HL560265,12,73593.0,0.19339575,DPD_90,0,79,64,64,69,54,12-02-2012,28-01-2024,517.06,1551.18,InForce,1,0.4,0.18597998,0.07702998,0.04170214,0.99015105,0.13333334,0.012380997,0.95566547,0.042800467,0
CUST594764,20,F,658,0.25493795,92632,0.8950112,37,10-01-2014 09:55,525476139,1037603771,Transfer Funds,Jocelyn,Perry,jocelyn.perry@walsh-huber.com,112261,HL426857,12,186185.0,0.08071548,DPD_90,0,NA,NA,NA,NA,NA,06-09-2009,22-08-2021,1299.27,3897.81,Overdue,1,0.044444446,0.51216024,0.23525628,0.00197001,0.9056555,0.13333334,0.27446055,0.17859635,0.25035754,1
CUST305070,55,M,366,0.5502413,116731,0.95298964,7,03-08-2009 03:38,2599887850,8773615865,Pay with Tap,Dale,Moore,dale.moore@gilbert.net,112260,HL551585,15,68274.0,0.11426555,DPD_30,0,87,76,52,92,78,28-11-2019,11-09-2034,382.58,382.58,InForce,1,0.82222223,0.094420604,0.5416136,0.053912323,0.96478957,0.33333334,0.0,0.4099654,0.0071166004,0
CUST768887,26,M,476,0.2493546,97241,0.25433972,30,14-03-2010 16:26,3910536420,5942038379,Bill Payments,Michael,Hall,michael.hall@garcia.com,112257,HL892732,15,71272.0,0.121932514,DPD_30,1,NA,89,NA,66,NA,28-10-2012,10-10-2027,399.61,399.61,InForce,1,0.17777778,0.25178826,0.22946393,0.01190412,0.25221425,0.33333334,0.0069784224,0.46283853,0.011635459,0
CUST219645,38,M,771,0.92347324,92494,0.16453096,89,15-03-2015 04:16,1596449396,3348344054,Transaction Statements,Patrick,Bruce,patrick.bruce@henson.org,112264,HL441567,20,350485.0,0.08045979,DPD_30,0,NA,78,65,94,87,16-01-2008,22-12-2027,1472.18,1472.18,InForce,1,0.44444445,0.6738197,0.9288165,0.0016725688,0.16061543,0.6666667,0.65690047,0.17683306,0.29623872,0
CUST220402,63,M,588,0.26589173,98649,0.54949677,42,19-05-2017 08:19,2706889691,9607922433,Customize Card,Sean,King,sean.king@dougherty.biz,112258,HL388771,12,116362.0,0.07188486,DPD_30,1,NA,NA,98,76,40,08-09-2008,24-08-2020,811.58,811.58,Overdue,1,1.0,0.41201717,0.24662009,0.014938884,0.5532543,0.13333334,0.11193408,0.1176984,0.12095047,0
CUST689781,43,F,697,0.70783985,96666,0.6768759,65,03-01-2014 05:44,5650582702,3826592144,Transaction Statements,Shari,Maldonado,shari.maldonado@cline.com,112261,HL243518,25,353168.0,0.098377995,DPD_60,0,47,83,NA,NA,NA,15-03-2008,02-12-2033,1191.81,2383.62,InForce,1,0.5555556,0.56795424,0.7051114,0.010664782,0.68317235,1.0,0.66314566,0.30040115,0.22184338,1
CUST316296,46,F,353,0.028170139,109881,0.75311524,17,06-07-2014 16:52,1222380877,310852864,Help Center,Regina,Robinson,regina.robinson@humphrey-brooks.com,112263,HL408872,10,182866.0,0.085013784,DPD_30,1,NA,NA,45,NA,NA,11-01-2013,30-12-2022,1530.42,1530.42,InForce,1,0.62222224,0.07582261,0.0,0.039148025,0.76093125,0.0,0.26673496,0.20823847,0.3116925,1
CUST537213,21,F,330,0.6880198,96310,0.5543166,86,11-01-2014 00:26,6970514612,838712666,Help Center,Bailey,West,bailey.west@valdez-blair.info,112260,HL139128,20,475770.0,0.10235902,DPD_60,0,NA,NA,46,58,NA,15-02-2009,21-01-2029,2002.82,4005.64,InForce,1,0.06666667,0.042918455,0.68454945,0.009897469,0.5581702,0.6666667,0.9485254,0.3278552,0.43704242,1
CUST8765,30,M,888,0.765425,555675,0.987512,76,20-12-2015 06:34,43214568,5678962134,Easy Lock,Thejas,Gurki,thejas.gurki@gmail.com,112262,HL567542,10,217865.0,0.0548178,DPD_30,0,NA,NA,65,NA,NA,18-04-2015,02-12-2025,2544.61,0.0,InForce,1,0.26666668,0.8412017,0.76485217

In [0]:
non_defaulters_data = merged_customer_home_loan_scaled_data.where(merged_customer_home_loan_scaled_data.home_loan_defaulters == 0)
display(non_defaulters_data)

customer_id,age,gender,fico_scores,wallet_share,income_estimate,dti_ratio,average_time_spent(minutes),last_login,phone,ssn,action_performed,fname,lname,email,zip,home_loan_id,tenure,orig_amt,rate,dpd,ltv,soi_cohort_employee,soi_cohort_online_heavy,soi_cohort_workplace,soi_cohort_millitary_banking,soi_cohort_student,orig_dt,maturity_dt,sch_amt,amount_owed,loan_status,home_loan_defaulters,age_scaled,fico_scores_scaled,wallet_share_scaled,income_estimate_scaled,dti_ratio_scaled,tenure_scaled,orig_amt_scaled,rate_scaled,sch_amt_scaled,"gender_mapped (M:0,F:1)"
CUST301488,36,M,428,0.2144166,112882,0.8747584,81,08-07-2017 23:02,7793364908,8887475444,Pay with Tap,James,Rogers,james.rogers@olson.com,112259,HL199671,25,293295.0,0.19137831,DPD_0,0,58,NA,NA,NA,NA,05-02-2015,01-05-2040,1001.3,0.0,InForce,0,0.4,0.18311875,0.19321811,0.045616295,0.88499904,1.0,0.5237797,0.94175273,0.1712921,0
CUST235925,36,F,564,0.1602604,101331,0.6702551,48,27-12-2015 04:12,2377129120,7323084732,Pay with Tap,Margaret,Vasquez,margaret.vasquez@day.com,112263,HL868849,25,341897.0,0.0902915,DPD_0,0,NA,88,68,87,40,29-07-2008,28-06-2033,1152.61,0.0,InForce,0,0.4,0.37768242,0.13703471,0.020719593,0.6764196,1.0,0.6369102,0.2446348,0.21144176,1
CUST768307,41,M,743,0.08002886,97891,0.66328233,80,15-08-2013 10:16,4589178568,1425823974,Zelle QR Code,Derek,Rivera,derek.rivera@greene.com,112256,HL163604,10,417023.0,0.10936252,DPD_0,0,NA,68,NA,84,NA,26-01-2011,13-01-2021,3494.39,0.0,Matured,0,0.51111114,0.63376254,0.053799924,0.013305113,0.6693078,0.0,0.81178045,0.37615296,0.832826,0
CUST854589,18,F,702,0.94196665,112343,0.42973027,39,29-01-2013 18:17,650494074,8475385450,Help Center,Sarah,Campbell,sarah.campbell@reese.com,112257,HL122253,12,142169.0,0.0677801,DPD_0,1,NA,NA,60,NA,74,02-03-2009,15-02-2021,991.33,0.0,Matured,0,0.0,0.5751073,0.94800216,0.044454552,0.43110064,0.13333334,0.17200485,0.08939101,0.16864659,1
CUST288657,22,F,649,0.41102615,113523,0.6269411,15,08-04-2019 08:18,9728883573,267725099,Pay with Tap,Amanda,Lara,amanda.lara@downs-young.com,112259,HL464229,12,327103.0,0.18228517,DPD_0,1,62,NA,99,57,52,25-05-2016,05-10-2028,2296.66,0.0,InForce,0,0.08888889,0.49928468,0.39718726,0.04699789,0.63224226,0.13333334,0.60247433,0.87904435,0.51501197,1
CUST301895,35,F,315,0.9920883,97164,0.7906885,20,02-04-2011 04:59,5236363153,6809263436,Bill Payments,Jennifer,Jones,jennifer.jones@taylor-anderson.com,112262,HL527310,25,267019.0,0.111554414,DPD_0,0,NA,NA,78,NA,NA,06-08-2015,07-05-2040,902.57,0.0,InForce,0,0.37777779,0.021459227,1.0,0.011738157,0.79925346,1.0,0.46261725,0.39126876,0.1450944,1
CUST747228,28,F,724,0.87314516,108038,0.09236621,62,11-04-2018 18:35,8995232999,1918835875,Transaction Statements,Beverly,Gonzalez,beverly.gonzalez@thomas.com,112256,HL872988,15,385598.0,0.080322176,DPD_0,1,NA,92,NA,70,NA,21-07-2019,07-02-2034,2155.21,0.0,InForce,0,0.22222222,0.60658085,0.8766045,0.035175674,0.08701229,0.33333334,0.73863274,0.17588405,0.47747865,1
CUST626632,45,F,645,0.77135015,94656,0.93964756,36,15-08-2012 02:27,2333931362,2295650717,Zelle QR Code,Lisa,Gibbs,lisa.gibbs@townsend.com,112255,HL195807,10,490995.0,0.15851834,DPD_0,0,NA,NA,NA,89,NA,10-11-2018,28-10-2028,4124.41,0.0,InForce,0,0.6,0.49356222,0.7709991,0.0063324836,0.95118153,0.0,0.9839645,0.71514285,1.0,1
CUST648137,54,F,470,0.8984357,112199,0.67008674,15,20-12-2011 07:34,5916176340,3581714686,Help Center,Kiara,Duarte,kiara.duarte@bradford.org,112260,HL431682,25,350086.0,0.090155594,DPD_0,0,NA,NA,NA,40,99,21-04-2018,21-03-2043,1180.2,0.0,InForce,0,0.8,0.24320458,0.90284175,0.044144176,0.6762479,1.0,0.6559717,0.24369757,0.21876267,1
CUST164089,32,M,775,0.13027395,96535,0.82738477,48,11-04-2010 07:09,7300372011,6068181069,Easy Lock,Donald,Frey,donald.frey@stevens.net,112257,HL681200,20,84221.0,0.13664018,DPD_0,0,63,81,94,96,NA,07-11-2011,13-10-2031,355.76,0.0,InForce,0,0.31111112,0.6795422,0.10592581,0.010382428,0.8366811,0.6666667,0.037119713,0.56426597,0.0,0


In [0]:
non_defaulters_data = non_defaulters_data.withColumnRenamed("customer_id", "customer_id_non_defaulters")
display(non_defaulters_data)

customer_id_non_defaulters,age,gender,fico_scores,wallet_share,income_estimate,dti_ratio,average_time_spent(minutes),last_login,phone,ssn,action_performed,fname,lname,email,zip,home_loan_id,tenure,orig_amt,rate,dpd,ltv,soi_cohort_employee,soi_cohort_online_heavy,soi_cohort_workplace,soi_cohort_millitary_banking,soi_cohort_student,orig_dt,maturity_dt,sch_amt,amount_owed,loan_status,home_loan_defaulters,age_scaled,fico_scores_scaled,wallet_share_scaled,income_estimate_scaled,dti_ratio_scaled,tenure_scaled,orig_amt_scaled,rate_scaled,sch_amt_scaled,"gender_mapped (M:0,F:1)"
CUST301488,36,M,428,0.2144166,112882,0.8747584,81,08-07-2017 23:02,7793364908,8887475444,Pay with Tap,James,Rogers,james.rogers@olson.com,112259,HL199671,25,293295.0,0.19137831,DPD_0,0,58,NA,NA,NA,NA,05-02-2015,01-05-2040,1001.3,0.0,InForce,0,0.4,0.18311875,0.19321811,0.045616295,0.88499904,1.0,0.5237797,0.94175273,0.1712921,0
CUST235925,36,F,564,0.1602604,101331,0.6702551,48,27-12-2015 04:12,2377129120,7323084732,Pay with Tap,Margaret,Vasquez,margaret.vasquez@day.com,112263,HL868849,25,341897.0,0.0902915,DPD_0,0,NA,88,68,87,40,29-07-2008,28-06-2033,1152.61,0.0,InForce,0,0.4,0.37768242,0.13703471,0.020719593,0.6764196,1.0,0.6369102,0.2446348,0.21144176,1
CUST768307,41,M,743,0.08002886,97891,0.66328233,80,15-08-2013 10:16,4589178568,1425823974,Zelle QR Code,Derek,Rivera,derek.rivera@greene.com,112256,HL163604,10,417023.0,0.10936252,DPD_0,0,NA,68,NA,84,NA,26-01-2011,13-01-2021,3494.39,0.0,Matured,0,0.51111114,0.63376254,0.053799924,0.013305113,0.6693078,0.0,0.81178045,0.37615296,0.832826,0
CUST854589,18,F,702,0.94196665,112343,0.42973027,39,29-01-2013 18:17,650494074,8475385450,Help Center,Sarah,Campbell,sarah.campbell@reese.com,112257,HL122253,12,142169.0,0.0677801,DPD_0,1,NA,NA,60,NA,74,02-03-2009,15-02-2021,991.33,0.0,Matured,0,0.0,0.5751073,0.94800216,0.044454552,0.43110064,0.13333334,0.17200485,0.08939101,0.16864659,1
CUST288657,22,F,649,0.41102615,113523,0.6269411,15,08-04-2019 08:18,9728883573,267725099,Pay with Tap,Amanda,Lara,amanda.lara@downs-young.com,112259,HL464229,12,327103.0,0.18228517,DPD_0,1,62,NA,99,57,52,25-05-2016,05-10-2028,2296.66,0.0,InForce,0,0.08888889,0.49928468,0.39718726,0.04699789,0.63224226,0.13333334,0.60247433,0.87904435,0.51501197,1
CUST301895,35,F,315,0.9920883,97164,0.7906885,20,02-04-2011 04:59,5236363153,6809263436,Bill Payments,Jennifer,Jones,jennifer.jones@taylor-anderson.com,112262,HL527310,25,267019.0,0.111554414,DPD_0,0,NA,NA,78,NA,NA,06-08-2015,07-05-2040,902.57,0.0,InForce,0,0.37777779,0.021459227,1.0,0.011738157,0.79925346,1.0,0.46261725,0.39126876,0.1450944,1
CUST747228,28,F,724,0.87314516,108038,0.09236621,62,11-04-2018 18:35,8995232999,1918835875,Transaction Statements,Beverly,Gonzalez,beverly.gonzalez@thomas.com,112256,HL872988,15,385598.0,0.080322176,DPD_0,1,NA,92,NA,70,NA,21-07-2019,07-02-2034,2155.21,0.0,InForce,0,0.22222222,0.60658085,0.8766045,0.035175674,0.08701229,0.33333334,0.73863274,0.17588405,0.47747865,1
CUST626632,45,F,645,0.77135015,94656,0.93964756,36,15-08-2012 02:27,2333931362,2295650717,Zelle QR Code,Lisa,Gibbs,lisa.gibbs@townsend.com,112255,HL195807,10,490995.0,0.15851834,DPD_0,0,NA,NA,NA,89,NA,10-11-2018,28-10-2028,4124.41,0.0,InForce,0,0.6,0.49356222,0.7709991,0.0063324836,0.95118153,0.0,0.9839645,0.71514285,1.0,1
CUST648137,54,F,470,0.8984357,112199,0.67008674,15,20-12-2011 07:34,5916176340,3581714686,Help Center,Kiara,Duarte,kiara.duarte@bradford.org,112260,HL431682,25,350086.0,0.090155594,DPD_0,0,NA,NA,NA,40,99,21-04-2018,21-03-2043,1180.2,0.0,InForce,0,0.8,0.24320458,0.90284175,0.044144176,0.6762479,1.0,0.6559717,0.24369757,0.21876267,1
CUST164089,32,M,775,0.13027395,96535,0.82738477,48,11-04-2010 07:09,7300372011,6068181069,Easy Lock,Donald,Frey,donald.frey@stevens.net,112257,HL681200,20,84221.0,0.13664018,DPD_0,0,63,81,94,96,NA,07-11-2011,13-10-2031,355.76,0.0,InForce,0,0.31111112,0.6795422,0.10592581,0.010382428,0.8366811,0.6666667,0.037119713,0.56426597,0.0,0


## INGESTING DATA INTO NEO4J CLOUD INSTANCE

In [0]:
defaulters_data.write\
  .format("org.neo4j.spark.DataSource")\
  .mode("Overwrite")\
  .option("authentication.type", "basic")\
  .option("url", "bolt://52.90.57.219:7687")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", "exhaust-highlines-comma")\
  .option("labels", "Defaulters")\
  .option("node.keys","customer_id_defaulters")\
  .save()

In [0]:
non_defaulters_data.write\
  .format("org.neo4j.spark.DataSource")\
  .mode("Overwrite")\
  .option("authentication.type", "basic")\
  .option("url", "bolt://52.90.57.219:7687")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", "exhaust-highlines-comma")\
  .option("labels", "Non Defaulters")\
  .option("node.keys","customer_id_non_defaulters")\
  .save()

### Building connection using credentials for graph projection

In [0]:
graph = Graph("bolt://52.90.57.219:7687",auth=("neo4j","exhaust-highlines-comma"))

#### Graph projection is already created. This cell needs to run only when creating a new graph projection. To debug or to create a graph with different name - remove %md from line 1
tx = graph.begin()
tx.evaluate( """
    CALL gds.graph.project(
    'myGraphVyome141_KNN_graph',
    {
        Defaulters: {
            properties: ['age_scaled',
 'fico_scores_scaled',
 'wallet_share_scaled',
 'income_estimate_scaled',
 'dti_ratio_scaled',
 'tenure_scaled',
 'orig_amt_scaled',
 'rate_scaled',
 'sch_amt_scaled']
        },
        
        `Non Defaulters`: {
            properties: ['age_scaled',
 'fico_scores_scaled',
 'wallet_share_scaled',
 'income_estimate_scaled',
 'dti_ratio_scaled',
 'tenure_scaled',
 'orig_amt_scaled',
 'rate_scaled',
 'sch_amt_scaled']
        }
    },
    '*'
)YIELD
  graphName,
  nodeProjection,
  nodeCount,
  relationshipProjection,
  relationshipCount;

""" )
graph.commit(tx)

#### Using Graph K Nearest Neighbours Algorithm to determine Potential Home Loan Defaulters by finding similarity scores between Defaulters and Non Defaulters

In [0]:
query = """
      CALL gds.alpha.knn.filtered.stream('myGraphVyome141_KNN_graph', {
    topK: 3,
    nodeProperties: ['age_scaled',
 'fico_scores_scaled',
 'wallet_share_scaled',
 'income_estimate_scaled',
 'dti_ratio_scaled',
 'tenure_scaled',
 'orig_amt_scaled',
 'rate_scaled',
 'sch_amt_scaled'],
   sourceNodeFilter: 'Defaulters',
   targetNodeFilter: 'Non Defaulters',
    // The following parameters are set to produce a deterministic result
    randomSeed: 1337,
    concurrency: 1,
    sampleRate: 0.75,
    deltaThreshold: 0.0
})
YIELD node1, node2, similarity

RETURN gds.util.asNode(node1).customer_id_defaulters AS customer_id_defaulters, gds.util.asNode(node2).customer_id_non_defaulters AS customer_id_non_defaulters, similarity
      
"""


df_similarity = spark.read.format("org.neo4j.spark.DataSource")\
    .option("url", "bolt://52.90.57.219:7687")\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "exhaust-highlines-comma")\
    .option("query", query) \
    .option("partitions", "1") \
    .load()

display(df_similarity)

customer_id_defaulters,customer_id_non_defaulters,similarity
CUST499509,CUST301488,0.8737530120382685
CUST499509,CUST164089,0.8557881718836708
CUST499509,CUST290194,0.8467684320120017
CUST594764,CUST807344,0.8748010506101452
CUST594764,CUST854589,0.8743478520087958
CUST594764,CUST469240,0.867096278435157
CUST305070,CUST864555,0.8988718973704601
CUST305070,CUST238356,0.8631531063939064
CUST305070,CUST807344,0.8567458447249418
CUST768887,CUST973003,0.9149361284716827


#### Writing Similarity scores to neo4j sandbox

In [0]:
df_similarity.write\
  .format("org.neo4j.spark.DataSource")\
  .mode("Overwrite")\
  .option("authentication.type", "basic")\
  .option("url", "bolt://52.90.57.219:7687")\
  .option("authentication.basic.username", "neo4j")\
  .option("authentication.basic.password", "exhaust-highlines-comma")\
  .option("relationship", "SIMILAR")\
  .option("relationship.save.strategy", "keys")\
  .option("relationship.source.labels", "Defaulters")\
  .option("relationship.source.node.keys", "customer_id_defaulters")\
  .option("relationship.source.save.mode", "overwrite")\
  .option("relationship.target.labels", "Non Defaulters")\
  .option("relationship.target.node.keys", "customer_id_non_defaulters")\
  .option("relationship.properties","similarity")\
  .option("relationship.target.save.mode", "overwrite")\
  .save()

#### Graph projection for degree centrality algorithm

##### Graph projection is already created. This cell needs to run only when creating a new graph projection. To debug or to create a graph with different name - remove %md from line 1

degree = graph.begin()
degree.evaluate("""CALL gds.graph.project(
  'myGraph_Vyome_Degree_Projection',
  ['Defaulters','Non Defaulters'],
  {
    SIMILAR: {
      orientation: 'REVERSE',
      properties: ['similarity']
    }
  }
)
""")
graph.commit(degree)

#### How many defaulters a non-defaulter shares similarities with is determined by Degree Centrality Score.

In [0]:
query = """CALL gds.degree.stream('myGraph_Vyome_Degree_Projection')
YIELD nodeId, score
where gds.util.asNode(nodeId).customer_id_non_defaulters is not null
RETURN gds.util.asNode(nodeId).customer_id_non_defaulters AS customer_id_non_defaulters, score AS similar_defaulter_score
"""
degree_results = spark.read.format("org.neo4j.spark.DataSource")\
    .option("url", "bolt://52.90.57.219:7687")\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "exhaust-highlines-comma")\
    .option("query", query) \
    .option("partitions", "1") \
    .load()

degree_results = degree_results.orderBy('similar_defaulter_score',ascending = False)
display(degree_results)


customer_id_non_defaulters,similar_defaulter_score
CUST807344,4.0
CUST469240,3.0
CUST854589,2.0
CUST238356,1.0
CUST254353,1.0
CUST301488,1.0
CUST164089,1.0
CUST675159,1.0
CUST599158,1.0
CUST660250,1.0


#### Graph Projection for Weighted Eigen Vector Centrality Algorithm

##### Graph projection is already created. This cell needs to run only when creating a new graph projection. To debug or to create a graph with different name - remove %md from line 1
eigen = graph.begin()
eigen.evaluate( """CALL gds.graph.project(
  'myGraph_Vyome_Eigen_Projection',
  ['Defaulters','Non Defaulters'],
  'SIMILAR',
  {
    relationshipProperties: 'similarity'
  }
)
""" )
graph.commit(eigen)

#### We are computing Eigenvector similarity to take the similarity percentage between a defaulter and non defaulter in consideration as well

In [0]:
query = """CALL gds.eigenvector.stream('myGraph_Vyome_Eigen_Projection',{relationshipWeightProperty : 'similarity',scaler :'L2Norm'})
YIELD nodeId, score
where gds.util.asNode(nodeId).customer_id_non_defaulters is not null
RETURN gds.util.asNode(nodeId).customer_id_non_defaulters AS customer_id_non_defaulters, score * 100 as eigen_similar_defaulter_score
"""
eigen_results = spark.read.format("org.neo4j.spark.DataSource")\
    .option("url", "bolt://52.90.57.219:7687")\
    .option("authentication.type", "basic")\
    .option("authentication.basic.username", "neo4j")\
    .option("authentication.basic.password", "exhaust-highlines-comma")\
    .option("query", query) \
    .option("partitions", "1") \
    .load()

eigen_results = eigen_results.orderBy('eigen_similar_defaulter_score',ascending = False)
display(eigen_results)


customer_id_non_defaulters,eigen_similar_defaulter_score
CUST807344,53.46651630700174
CUST469240,40.60327476480497
CUST854589,27.68744043606206
CUST973003,15.272817302773229
CUST864555,15.21061373159109
CUST747228,15.185455659620455
CUST648137,15.074831667723753
CUST301488,15.052090120438816
CUST649539,15.006923846743545
CUST235925,14.991808377150436
